In [1]:
# input_header = {
#     type: 'analyst',
#     setting: {
#         'fin_statement_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_Q1', '2019_Q2', ...]}, # 리스트 형태에 담아서 원하는 개수만큼 추출해 반환 (단, 분기별 코드만 인정)
#         'reports_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_Q1', '2019_Q2', ...]}, # 리스트 형태에 담아서 원하는 개수만큼 추출해 반환 (단, 분기별 코드만 인정)
#         'stock_price_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'index_price_setting': {'target_index': ['코스피', '코스피 200', ...], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'corp_rel_news_setting': {'tickers': ['005930', '005380', '...'], 'start_date': '20190101', end_date: '20190701'}, (단, YYYYMMDD만 인정)
#         'sector_analysis_setting': {'target_index': ['코스피', '코스피 200', ...], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#     }
# }

# input_header: {
#     type: 'trader',
#     setting: {
#         'stock_price_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'index_price_setting': {'target_index': ['코스피', '코스피 200', ...], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'corp_rel_news_setting': {'tickers': ['005930', '005380', '...'], 'start_date': '20190101', end_date: '20190701'}, (단, YYYYMMDD만 인정)
#         'pattern_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_01', '2019_02', ...]}, (단, 분기별 코드만 인정)
#         'price_estimation_setting': {'tickers':: ['005930', '005380', '...'], 'timespan': ['2019_01', '2019_02', ...]}
#     }
# }

# input_header: {
#     type: 'portfolio_manager',
#     setting: {
#         'stock_price_setting': {'tickers': ['005930', '005380', '...'], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'index_price_setting': {'target_index': ['코스피', '코스피 200', ...], 'timespan': ['2019_01', '2019_02', ...]}, (단, 월별 코드만 인정)
#         'current_portfolio_setting': {'start_date': '20190101', 'end_date': '20191231'}, (포트폴리오 구성 로그에 대해 start_date ~ end_date 사이의 정보를 반환)
#         'macro_econ_info_setting': {'timespan': ['2019_01', '2019_02', ...]}, (단, 분기별 코드만 인정)
#     }
# }

# input_header: {
#     type: 'macro_strategist',
#     setting: {
#         'macro_econ_info_setting': {'timespan': ['2019_01', '2019_02', ...]}, (단, 분기별 코드만 인정)
#         'intl_info_setting': {'start_date': '20190101', end_date: '20190701'}, (단, YYYYMMDD만 인정)
#     }
# }

In [2]:
import pandas as pd
import re

def df_to_json(df, orient='records', indent=4):
    try:
        # DataFrame을 JSON 문자열로 변환
        json_str = df.to_json(orient=orient, indent=indent, force_ascii=False)
        return json_str
    except Exception as e:
        print(f"Error converting DataFrame to JSON: {str(e)}")
        return None
    
def clean_text(text):
    text = re.sub(r'[.*?]', '', text)
    text = re.sub(r'\\[a-z]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def receive_from_gpt(input_text):
    # GPT에게 정보를 받는 함수
    return input_text

def send_to_gpt(output_text):
    # GPT에게 정보를 보내는 함수, 여기서 dict나 df를 json으로 변환
    output_json = output_text
    return output_json

In [3]:
from sub_func import fin_statement_info # ticker, year, quarter -> df
from sub_func import reports_info # ticker, year, quarter -> df
from sub_func import corp_rel_news_info # ticker, year, start_date, end_date -> df
from sub_func import intl_news_info # year, start_date, end_date -> df
from sub_func import macro_econ_info # year, start_date, end_date -> dict
from sub_func import stock_price_info # ticker, start_date, end_date -> df
from sub_func import index_price_info # sector, start_date, end_date -> df
from sub_func import pattern_info # ticker, date -> df
from sub_func import sector_analysis_info # sector, year, quarter -> dict

def price_estimation_info():
    return None 

def current_portfolio_info():
    return None

def portfolio_analysis_info():
    return None

In [13]:
input_header = {
    'type': 'analyst',
    'setting': {
        'fin_statement_setting': {'tickers': ['005930', '000660'], 'timespan': ['2023_Q1', '2023_Q2']},
        'reports_setting': {'tickers': ['005930', '000660'], 'timespan': ['2023_Q1', '2023_Q2']},
        'stock_price_setting': {'tickers': ['005930', '000660'], 'timespan': ['20230101', '20230401']}, 
        'index_price_setting': {'target_index': ['코스피', '코스피 200'], 'timespan': ['20230101', '20230401']}, 
        'corp_rel_news_setting': {'tickers': ['005930', '000660'], 'start_date': '20230101', 'end_date': '20230701'}, 
        'sector_analysis_setting': {'target_index': ['코스피', '코스피 200'], 'timespan': ['2023_Q1', '2023_Q2']}, 
        'macro_econ_info_setting': {'timespan': ['20190101', '20190301']},
        'intl_info_setting': {'timespan': ['20190101', '20190301']},
    }
}

In [14]:
# agent_settings 및 setting_to_function을 기반으로 정보를 가져와 infos_to_send에 담는 함수
gpt_agent_type = input_header['type']

# 각 에이전트 타입에 따른 유효한 설정 목록
agent_settings = {
    'analyst': [
        'fin_statement_setting', 'reports_setting', 'stock_price_setting',
        'index_price_setting', 'corp_rel_news_setting', 'sector_analysis_setting'
    ],
    'trader': [
        'stock_price_setting', 'index_price_setting', 'corp_rel_news_setting',
        'pattern_setting', 'price_estimation_setting'
    ],
    'portfolio_manager': [
        'stock_price_setting', 'index_price_setting', 'current_portfolio_setting',
        'macro_econ_info_setting'
    ],
    'macro_strategist': [   
        'macro_econ_info_setting', 'intl_info_setting'
    ]
}

# 설정에 해당하는 함수를 저장하는 딕셔너리
setting_to_function = {
    'fin_statement_setting': fin_statement_info,
    'reports_setting': reports_info,
    'stock_price_setting': stock_price_info,
    'index_price_setting': index_price_info,
    'corp_rel_news_setting': corp_rel_news_info,
    'pattern_setting': pattern_info,
    'price_estimation_setting': price_estimation_info,
    'current_portfolio_setting': current_portfolio_info,
    'macro_econ_info_setting': macro_econ_info,
    'intl_info_setting': intl_news_info,
    'sector_analysis_setting': sector_analysis_info
}

In [15]:
# # 에이전트 타입이 유효하지 않을 경우 처리
# if gpt_agent_type not in agent_settings:
#     print(f'GPT에게 전달받은 agent type이 유효하지 않습니다. 전달받은 agent type: {gpt_agent_type}')
#     raise ValueError(f'Invalid agent type: {gpt_agent_type}')

# # input_header에서 필요한 값들을 infos_to_send에 담기
# infos_to_send = {}
# setting_value = input_header['setting']

# # pattern, price estimation, current portfolio는 추가 필요
# for setting in setting_value:
#     try:
#         if setting == 'fin_statement_setting':
#             for ticker in setting_value[setting]['tickers']:
#                 for timespan in setting_value[setting]['timespan']:
#                     year, quarter = timespan.split('_')
#                     result = setting_to_function[setting](ticker, year, quarter)
#                     infos_to_send[f"{setting}_{ticker}_{timespan}"] = result

#         elif setting == 'reports_setting':
#             for ticker in setting_value[setting]['tickers']:
#                 for timespan in setting_value[setting]['timespan']:
#                     year, quarter = timespan.split('_')
#                     result = setting_to_function[setting](ticker, year, quarter)
                    
#                     columns = result.columns
#                     for c in range(len(columns)):
#                         inner_text = list(result[columns[c]])[0]
#                         inner_text = eval(inner_text)[0]
#                         inner_text = clean_text(inner_text)

#                     infos_to_send[f"{setting}_{ticker}_{timespan}"] = inner_text

#         elif setting == 'stock_price_setting':
#             for ticker in setting_value[setting]['tickers']:
#                 timespan_list = setting_value[setting]['timespan']
#                 start_date = timespan_list[0]
#                 end_date = timespan_list[-1]
#                 result = setting_to_function[setting](ticker, start_date, end_date)
#                 infos_to_send[f"{setting}_{ticker}_{start_date}_{end_date}"] = result

#         elif setting == 'index_price_setting':
#             for target_index in setting_value[setting]['target_index']:
#                 timespan_list = setting_value[setting]['timespan']
#                 start_date = timespan_list[0]
#                 end_date = timespan_list[-1]
#                 result = setting_to_function[setting](target_index, start_date, end_date)
#                 infos_to_send[f"{setting}_{target_index}_{start_date}_{end_date}"] = result

#         elif setting == 'corp_rel_news_setting':
#             for ticker in setting_value[setting]['tickers']:
#                 start_date = setting_value[setting]['start_date']
#                 end_date = setting_value[setting]['end_date']
#                 year = start_date[:4]  # 년도는 시작 날짜에서 추출
#                 result = setting_to_function[setting](ticker, year, start_date, end_date)
#                 infos_to_send[f"{setting}_{ticker}_{start_date}_{end_date}"] = result

#         elif setting == 'sector_analysis_setting':
#             for target_index in setting_value[setting]['target_index']:
#                 for timespan in setting_value[setting]['timespan']:
#                     year, quarter = timespan.split('_')
#                     result = setting_to_function[setting](target_index, year, quarter)
#                     infos_to_send[f"{setting}_{target_index}_{timespan}"] = result

#         elif setting == 'macro_econ_info':
#             timespan_list = setting_value[setting]
#             for timespan in timespan_list['timespan']:
#                 year = timespan[0].split('_')[0]
#                 start_date = timespan[0]
#                 end_date = timespan[-1]
#                 result = setting_to_function[setting](year, start_date, end_date)
#                 infos_to_send[f"{setting}_{start_date}_{end_date}"] = result

#         elif setting == 'intl_news_info':
#             timespan_list = setting_value[setting]
#             for timespan in timespan_list['timespan']:
#                 year = timespan[0].split('_')[0]
#                 start_date = timespan[0]
#                 end_date = timespan[-1]
#                 result = setting_to_function[setting](year, start_date, end_date)
#                 infos_to_send[f"{setting}_{start_date}_{end_date}"] = result

#     except Exception as e:
#         print(f"Error processing {setting}: {str(e)}")

# for info in infos_to_send:
#     values = infos_to_send[info]

#     if type(values) == pd.DataFrame:
#         infos_to_send[info] = df_to_json(infos_to_send[info])

In [21]:
def pipeline_main(input_header):
    # 에이전트 타입이 유효하지 않을 경우 처리
    if gpt_agent_type not in agent_settings:
        print(f'GPT에게 전달받은 agent type이 유효하지 않습니다. 전달받은 agent type: {gpt_agent_type}')
        raise ValueError(f'Invalid agent type: {gpt_agent_type}')

    # input_header에서 필요한 값들을 infos_to_send에 담기
    infos_to_send = {}
    setting_value = input_header['setting']

    # pattern, price estimation, current portfolio는 추가 필요
    for setting in setting_value:
        try:
            if setting == 'fin_statement_setting':
                for ticker in setting_value[setting]['tickers']:
                    for timespan in setting_value[setting]['timespan']:
                        year, quarter = timespan.split('_')
                        result = setting_to_function[setting](ticker, year, quarter)
                        infos_to_send[f"{setting}_{ticker}_{timespan}"] = result

            elif setting == 'reports_setting':
                for ticker in setting_value[setting]['tickers']:
                    for timespan in setting_value[setting]['timespan']:
                        year, quarter = timespan.split('_')
                        result = setting_to_function[setting](ticker, year, quarter)
                        
                        columns = result.columns
                        for c in range(len(columns)):
                            inner_text = list(result[columns[c]])[0]
                            inner_text = eval(inner_text)[0]
                            inner_text = clean_text(inner_text)

                        infos_to_send[f"{setting}_{ticker}_{timespan}"] = inner_text

            elif setting == 'stock_price_setting':
                for ticker in setting_value[setting]['tickers']:
                    timespan_list = setting_value[setting]['timespan']
                    start_date = timespan_list[0]
                    end_date = timespan_list[-1]
                    result = setting_to_function[setting](ticker, start_date, end_date)
                    infos_to_send[f"{setting}_{ticker}_{start_date}_{end_date}"] = result

            elif setting == 'index_price_setting':
                for target_index in setting_value[setting]['target_index']:
                    timespan_list = setting_value[setting]['timespan']
                    start_date = timespan_list[0]
                    end_date = timespan_list[-1]
                    result = setting_to_function[setting](target_index, start_date, end_date)
                    infos_to_send[f"{setting}_{target_index}_{start_date}_{end_date}"] = result

            elif setting == 'corp_rel_news_setting':
                for ticker in setting_value[setting]['tickers']:
                    start_date = setting_value[setting]['start_date']
                    end_date = setting_value[setting]['end_date']
                    year = start_date[:4]  # 년도는 시작 날짜에서 추출
                    result = setting_to_function[setting](ticker, year, start_date, end_date)
                    infos_to_send[f"{setting}_{ticker}_{start_date}_{end_date}"] = result

            elif setting == 'sector_analysis_setting':
                for target_index in setting_value[setting]['target_index']:
                    for timespan in setting_value[setting]['timespan']:
                        year, quarter = timespan.split('_')
                        result = setting_to_function[setting](target_index, year, quarter)
                        infos_to_send[f"{setting}_{target_index}_{timespan}"] = result

            elif setting == 'macro_econ_info':
                timespan_list = setting_value[setting]
                for timespan in timespan_list['timespan']:
                    year = timespan[0].split('_')[0]
                    start_date = timespan[0]
                    end_date = timespan[-1]
                    result = setting_to_function[setting](year, start_date, end_date)
                    infos_to_send[f"{setting}_{start_date}_{end_date}"] = result

            elif setting == 'intl_news_info':
                timespan_list = setting_value[setting]
                for timespan in timespan_list['timespan']:
                    year = timespan[0].split('_')[0]
                    start_date = timespan[0]
                    end_date = timespan[-1]
                    result = setting_to_function[setting](year, start_date, end_date)
                    infos_to_send[f"{setting}_{start_date}_{end_date}"] = result

        except Exception as e:
            print(f"Error processing {setting}: {str(e)}")

    for info in infos_to_send:
        values = infos_to_send[info]

        if type(values) == pd.DataFrame:
            infos_to_send[info] = df_to_json(infos_to_send[info])

    return infos_to_send

In [22]:
pipeline_main(input_header)

{'fin_statement_setting_005930_2023_Q1': '[\n    {\n        "Stock Price":64000,\n        "PER":242.643263813,\n        "PBR":1.061886335,\n        "ROE":0.0043763273,\n        "Profit Growth Rate (%)":-86.0957905196,\n        "CAGR (%)":-0.8609579052\n    }\n]',
 'fin_statement_setting_005930_2023_Q2': '[\n    {\n        "Stock Price":72200,\n        "PER":250.072843016,\n        "PBR":1.2006895981,\n        "ROE":0.0048013594,\n        "Profit Growth Rate (%)":-84.4706614811,\n        "CAGR (%)":-0.8447066148\n    }\n]',
 'fin_statement_setting_000660_2023_Q1': '[\n    {\n        "Stock Price":88600,\n        "PER":-24.9472960993,\n        "PBR":1.0572456886,\n        "ROE":-0.0423791694,\n        "Profit Growth Rate (%)":-230.3875380246,\n        "CAGR (%)":-2.3038753802\n    }\n]',
 'fin_statement_setting_000660_2023_Q2': '[\n    {\n        "Stock Price":115200,\n        "PER":-28.0684346762,\n        "PBR":1.4595763844,\n        "ROE":-0.0520006335,\n        "Profit Growth Rate (%

In [ ]:
cp = {
    
    
}